In [57]:
import sqlite3
import pandas as pd
import glob
import subprocess, os
import glob

In [4]:
'''
1. read file of cb_multios_execs
2. create dictionary k (original), v (patched)
3. cmd: bindiff orig.BinExport patched.BinExport
4. analysis (to get diff function names)
'''

'\n1. read file of cb_multios_execs\n2. create dictionary k (original), v (patched)\n3. cmd: bindiff orig.BinExport patched.BinExport\n4. analysis (to get diff function names)\n'

In [5]:
files = sorted([file for file in glob.glob('/home/user/Desktop/cbmultios_BinExports/*.BinExport')])

In [6]:
def make_pairs(files):
    originals = [file for file in files if "_patched" not in file]
    patched = [(file.replace('.BinExport', '_patched.BinExport')) for file in originals]
    pairs = dict(zip(originals, patched))
    #pairs = dict(zip(files[::2], files[1::2]))
    return pairs

In [7]:
def make_bindiff(original, patched):
    output_dir = '/home/user/Desktop/cbmultios_BinExports/bindiffs/'
    full_path = output_dir + original.split('/')[-1].replace('.BinExport','') + '_vs_' + patched.split('/')[-1].replace('.BinExport','') + '.BinDiff'
    cmd = ['/usr/bin/bindiff', original, patched, '--output_dir='+ output_dir]
    subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    return full_path

In [8]:
def bindiff_to_csv(bindiff_diff):
    print('bindiff to csv: ', bindiff_diff)
    #bindiff_diff = bindiff_diff.replace('.BinExport','')
    db = sqlite3.connect(bindiff_diff)
    cursor = db.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    try:
        fun_table = tables[4]
        #table = pd.read_sql_query("SELECT * from %s" % fun_table, db)
        table = pd.read_sql_query("SELECT * from %s" % fun_table, db)

        output_dir = '/home/user/Desktop/cbmultios_BinExports/bindiffs/funtables/'
        name = output_dir + bindiff_diff.split('/')[-1] + '_funtable.csv'
        table.to_csv(name, index_label='index')
        cursor.close()
        db.close()
        return name

    except:
        return 'error'

In [66]:
def get_fcn(name_sample_addr):
    try:
        index = fun_df.index[fun_df['sample_name_addr'] == name_sample_addr].tolist()[0]
        fun_name = fun_df.iloc[index]['fun_name']
    except:
        fun_name='not_found'
    return fun_name


In [68]:
def coorelate_fun_names(table_path):
    base_dir = '/home/user/Desktop/cbmultios_BinExports/resources/final_dfs_csvs/'
    bindiff_df = pd.read_csv(table_path, index_col=[0])
    name = list(table_path.split('/')[-1].replace('_funtable.csv','').replace('.BinDiff','').split('_vs_'))
    #index,id,address1,address2,similarity,confidence,
    #flags,algorithm,evaluate,commentsported,basicblocks,edges,instructions
    bindiff_df['original_sample'] = name[0] #get original sample name
    bindiff_df['patched_sample'] = name[1].replace('_funtable.csv', '') #get patched sample name

    bindiff_df['original_sample_addr'] = bindiff_df['original_sample'] + '_' + bindiff_df['address1'].apply(str)
    bindiff_df['patched_sample_addr'] = bindiff_df['patched_sample'] + '_' + bindiff_df['address2'].apply(str)

    bindiff_df['orig_fun_name'] = bindiff_df['original_sample_addr'].apply(lambda x: get_fcn(x))
    bindiff_df['patched_fun_name'] = bindiff_df['patched_sample_addr'].apply(lambda x: get_fcn(x))

    csv_name = base_dir + table_path.split('/')[-1].replace('.BinDiff','').replace('_funtable.csv','') + '_finaldf.csv'
    bindiff_df.to_csv(csv_name)
    return bindiff_df


In [39]:
base_dir = '/home/user/Desktop/cbmultios_BinExports/ghidra_function_data/'
fun_df = pd.read_csv(base_dir+'sample_functions_addresses.csv')
fun_df.columns = ['path', 'fun_name', 'addr']
fun_df['name'] = fun_df['path'].apply(lambda x: x.split('/')[-1])
fun_df['sample_name_addr'] = fun_df['name'] + '_' + fun_df['addr'].apply(str)

In [40]:
fun_df

,path,fun_name,addr,name,sample_name_addr
0,/home/user/Desktop/cb_multios_binaries_origina...,FUN_0804a030,134520880,3D_Image_Toolkit,3D_Image_Toolkit_134520880
1,/home/user/Desktop/cb_multios_binaries_origina...,cgc_pow,134520896,3D_Image_Toolkit,3D_Image_Toolkit_134520896
2,/home/user/Desktop/cb_multios_binaries_origina...,cgc__terminate,134520912,3D_Image_Toolkit,3D_Image_Toolkit_134520912
3,/home/user/Desktop/cb_multios_binaries_origina...,cgc_transmit,134520928,3D_Image_Toolkit,3D_Image_Toolkit_134520928
4,/home/user/Desktop/cb_multios_binaries_origina...,cgc_cos,134520944,3D_Image_Toolkit,3D_Image_Toolkit_134520944
...,...,...,...,...,...
94778,/home/user/Desktop/cb_multios_binaries_origina...,__gmon_start__,134541332,yolodex_patched,yolodex_patched_134541332
94779,/home/user/Desktop/cb_multios_binaries_origina...,__libc_start_main,134541336,yolodex_patched,yolodex_patched_134541336
94780,/home/user/Desktop/cb_multios_binaries_origina...,cgc_rint,134541340,yolodex_patched,yolodex_patched_134541340
94781,/home/user/Desktop/cb_multios_binaries_origina...,cgc_log10,134541344,yolodex_patched,yolodex_patched_134541344


In [12]:
pairs = make_pairs(files)

In [14]:
bindiffs = [make_bindiff(k, v) for k, v in pairs.items()]

In [ ]:
bindiff_fcn_paths = [bindiff_to_csv(bindiff) for bindiff in bindiffs]

In [69]:
final_dfs = [coorelate_fun_names(bindiff_fcn_path) for bindiff_fcn_path in bindiff_fcn_paths if bindiff_fcn_path != 'error']

In [43]:
len(final_dfs)

283

In [58]:
def collect_dfs(base_dir): 
    dfs = [pd.read_csv(file, index_col=[0]) for file in glob.glob(base_dir+ '*_finaldf.csv')]
    return dfs

In [70]:
base_dir = '/home/user/Desktop/cbmultios_BinExports/resources/final_dfs_csvs/'
frames = collect_dfs(base_dir)

In [71]:
len(frames)

283

In [72]:
final_combined = pd.concat(frames, axis=0)

In [76]:
final_combined.query('similarity < 1.0')

,id,address1,address2,similarity,confidence,flags,algorithm,evaluate,commentsported,basicblocks,edges,instructions,original_sample,patched_sample,original_sample_addr,patched_sample_addr,orig_fun_name,patched_fun_name
index,,,,,,,,,,,,,,,,,,
120,121,134554064,134554064,0.986724,0.989763,2,1,0,0,6,7,82,middleware_handshake,middleware_handshake_patched,middleware_handshake_134554064,middleware_handshake_patched_134554064,cgc_send_response,cgc_send_response
56,57,134527088,134527088,0.642040,0.963935,51,1,0,0,6,6,36,basic_emulator,basic_emulator_patched,basic_emulator_134527088,basic_emulator_patched_134527088,cgc_gb_load,cgc_gb_load
66,67,134527344,134527344,0.973705,0.993307,1,1,0,0,23,34,147,anagram_game,anagram_game_patched,anagram_game_134527344,anagram_game_patched_134527344,cgc_atree_add,cgc_atree_add
64,65,134526752,134526752,0.895304,0.993307,1,1,0,0,8,9,63,SFTSCBSISS,SFTSCBSISS_patched,SFTSCBSISS_134526752,SFTSCBSISS_patched_134526752,cgc_HandleReadRequest,cgc_HandleReadRequest
118,119,134536432,134536432,0.922022,0.992103,35,1,0,0,17,26,167,One_Vote,One_Vote_patched,One_Vote_134536432,One_Vote_patched_134536432,cgc_create_and_insert_vote,cgc_create_and_insert_vote
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,85,134536832,134536832,0.992626,0.992761,18,1,0,0,37,49,274,Accel,Accel_patched,Accel_134536832,Accel_patched_134536832,cgc_handle_op_median,cgc_handle_op_median
96,97,134530400,134530400,0.992600,0.992803,2,1,0,0,40,68,183,COLLIDEOSCOPE,COLLIDEOSCOPE_patched,COLLIDEOSCOPE_134530400,COLLIDEOSCOPE_patched_134530400,cgc_eval_ast_node,cgc_eval_ast_node
100,101,134531328,134531344,0.813328,0.942676,51,1,0,0,14,18,68,COLLIDEOSCOPE,COLLIDEOSCOPE_patched,COLLIDEOSCOPE_134531328,COLLIDEOSCOPE_patched_134531344,cgc_insert_variable,cgc_insert_variable


In [74]:
final_combined.to_csv('/home/user/Desktop/cbmultios_BinExports/resources/final_combined/final_combined.csv')